In [3]:
# Cell 1: Imports and Setup
import requests
import zipfile
import io
import pandas as pd
import numpy as np
import os

print("--- Notebook 01: Per-Trial Preprocessing ---")
# This will create the 'data' folder in the parent directory
os.makedirs('../data', exist_ok=True)

# Cell 2: Download and Extract Dataset
url = "https://springernature.figshare.com/articles/dataset/A_multisensor_dataset_of_south_asian_post_graduate_students_working_on_mental_rotation_tasks/28120670/files/51439640"
print("Downloading dataset...")
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
print("Dataset extracted.")

# Cell 3: Load all IVT and PSY files
all_ivt_data, all_psy_data = [], []
for pid in range(1, 39):
    try:
        with zip_file.open(f"STData/{pid}/{pid}_IVT.csv") as f:
            df_ivt = pd.read_csv(f)
            df_ivt['participant_id'] = pid
            all_ivt_data.append(df_ivt)
        with zip_file.open(f"STData/{pid}/{pid}_PSY.csv") as f:
            df_psy = pd.read_csv(f)
            df_psy['participant_id'] = pid
            all_psy_data.append(df_psy)
    except KeyError:
        print(f"Warning: Data not found for participant {pid}. Skipping.")

df_ivt_master = pd.concat(all_ivt_data, ignore_index=True)
df_psy_master = pd.concat(all_psy_data, ignore_index=True)

# Cell 4: Slice IVT data into trials
all_trials_data = []
for i, trial in df_psy_master.iterrows():
    p_ivt = df_ivt_master[df_ivt_master['participant_id'] == trial['participant_id']]
    trial_ivt = p_ivt[
        (p_ivt['UnixTime'] >= trial['routineStart']) &
        (p_ivt['UnixTime'] <= trial['routineEnd'])
    ].copy()
    trial_ivt['trial_id'] = trial['Key']
    all_trials_data.append(trial_ivt)

df_trials = pd.concat(all_trials_data, ignore_index=True)
print(f"Data sliced into {df_trials['trial_id'].nunique()} unique trials.")


# Cell 5: Clean, Separate, and Save
df_fix = df_trials[df_trials['Fixation Index'].notna()].copy()
df_sac = df_trials[df_trials['Saccade Index'].notna()].copy()

df_fix.to_csv('../data/processed_fixations_per_trial.csv', index=False)
df_sac.to_csv('../data/processed_saccades_per_trial.csv', index=False)
print("\n✅ Preprocessed per-trial data saved to the 'data' folder.")

--- Notebook 01: Per-Trial Preprocessing ---
Dataset extracted.


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10724\3529246063.py:25: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ivt = pd.read_csv(f)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10724\3529246063.py:25: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ivt = pd.read_csv(f)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10724\3529246063.py:25: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ivt = pd.read_csv(f)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10724\3529246063.py:25: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ivt = pd.read_csv(f)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10724\3529246063.py:25: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ivt = pd.read_csv(f)
C:\Users\ADMIN\AppData\Local\T

Data sliced into 46 unique trials.

✅ Preprocessed per-trial data saved to the 'data' folder.
